In [1]:
import pandas as pd
import numpy as np
import json
import pyperclip

In [2]:
def replace_layer(layers, old_layer_name, new_layer_arn):
    layers = layers.split(' ')
    new_layers = ' '.join([new_layer_arn if old_layer_name == layer.split(':')[-2] else layer for layer in layers])
    return new_layers

In [3]:
def generate_cmd(function_name, layers):
    return f'aws lambda update-function-configuration --function-name {function_name} --layers {layers} >> Downloads/aws_log.txt'

In [4]:
def cli_update_layers(functions_json_file, replacements, copy_to_clipboard=True):
    with open(functions_json_file) as json_file:
        functions_data = json.load(json_file)['Functions']
        
    for function in functions_data:
        try:
            function['Layers'] = " ".join([layer['Arn'] for layer in function['Layers']])
        except:
            pass
    
    _df = pd.DataFrame(functions_data)
    _df = _df[~pd.isna(_df.Layers)]
    
    for key in replacements:
        old_layer_name = key
        new_layer_arn = replacements[key]
        _df.Layers = _df.apply(lambda x: replace_layer(x.Layers, old_layer_name, new_layer_arn), axis=1)
    
    _df['cmd'] = _df.apply(lambda x: generate_cmd(x.FunctionName, x.Layers), axis=1)
    
    cmd = ';'.join(_df['cmd'].values.tolist())
    
    if copy_to_clipboard:
        pyperclip.copy(cmd)

    return cmd

In [5]:
def load_layer_latest_arn(layers_json_file):
    with open(layers_json_file) as json_file:
        layers_data = json.load(json_file)['Layers']
        
    layer_latest_arn = {}
    
    for layer in layers_data:
        name = layer['LayerName']
        arn = layer['LatestMatchingVersion']['LayerVersionArn']
        layer_latest_arn[name] = arn
        
    return layer_latest_arn

In [6]:
# aws lambda list-functions --region ap-east-1 --output json > Downloads/aws_lambda_functions.json
functions_file = 'aws_lambda_functions.json'

In [7]:
# aws lambda list-layers --region ap-east-1 --output json > Downloads/aws_lambda_layers.json
layers_file = 'aws_lambda_layers.json'
layer_latest_arn = load_layer_latest_arn(layers_file)

In [8]:
layer_latest_arn

{'aws-data-wrangler': 'arn:aws:lambda:ap-east-1:238911357348:layer:aws-data-wrangler:1',
 'gspread-plus': 'arn:aws:lambda:ap-east-1:238911357348:layer:gspread-plus:2',
 'httplib2': 'arn:aws:lambda:ap-east-1:238911357348:layer:httplib2:1',
 'kam-credentials': 'arn:aws:lambda:ap-east-1:238911357348:layer:kam-credentials:4',
 'pandas_numpy_cp39': 'arn:aws:lambda:ap-east-1:238911357348:layer:pandas_numpy_cp39:1',
 'pydrive': 'arn:aws:lambda:ap-east-1:238911357348:layer:pydrive:1',
 'tenacity': 'arn:aws:lambda:ap-east-1:238911357348:layer:tenacity:1'}

In [9]:
replacements = {
    'kam-credentials': layer_latest_arn['kam-credentials'],
    'aws-data-wrangler': layer_latest_arn['pandas_numpy_cp39'],
    'gspread': layer_latest_arn['gspread-plus'],
    'gspread-plus': layer_latest_arn['gspread-plus']
}

In [10]:
cmd = cli_update_layers(functions_file, replacements)